<a href="https://colab.research.google.com/github/uhyozzy/Project/blob/master/Predicting%20Road%20Flooding%20Risk/Model/%EB%8B%A4%EB%B3%80%EC%9D%B8%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%AA%A8%EB%8D%B8_%EB%B0%8F_%EC%98%88%EC%B8%A1(1)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **■ 강수량과 하수관로 비율 선형회귀 학습 후 예측**

# [1] TensorFlwo를 이용한 **다변인 선형회귀 모델**

참고1 : https://www.youtube.com/watch?v=9Ur7bbZtoTc 혹은 https://ndb796.tistory.com/126

참고2 : https://youtu.be/ve6gtpZV83E

- 다변인 선형회귀는 모델에 영향을 미치는 변인이 여러 개 일 때 사용하는 모델임. 현재 우리의 데이터에서는 변인이 '강수량', '지형', '지역환경이므로 이 모든 변인이 '하수관로비율'에 영향을 미친다고 감안해야 함.
- 학습률을 0.000005로 설정하여 되도록 짧은 시간에 정확한 결과가 나오도록 구현

**Base**

In [1]:
import tensorflow as tf
import numpy as np
from pandas.io.parsers import read_csv

In [2]:
# 파일을 쉼표로 구분하여 읽어 데이터를 DataFrame 형태로 불러옴
data_origin = read_csv('/content/강수량,하수관로.csv', sep=',')

In [3]:
# 결측치 확인
missing_values = data_origin.isnull().sum()
print(missing_values)

1hr 최대 강수량(mm)      537
일평균 강수량(mm)         537
[MAX] 하수관로 비율(%)    688
dtype: int64


In [4]:
# 결측치가 있는 행 제거한 후 확인
data = data_origin.dropna()
missing_values2 = data.isnull().sum()
print(missing_values2)

1hr 최대 강수량(mm)      0
일평균 강수량(mm)         0
[MAX] 하수관로 비율(%)    0
dtype: int64


In [5]:
# DataFrame을 NumPy 배열로 변환하고 데이터 타입을 float32로 지정
xy = np.array(data, dtype=np.float32)

In [6]:
# 2개의 변인을 입력을 받음
x_data = xy[:, 0:2]

In [7]:
# 하수관로(MAX) 값을 입력 받음
# 출력 데이터로 사용할 마지막 열의 데이터를 추출
y_data = xy[:, [-1]]

In [8]:
# 입력 데이터를 위한 TensorFlow placeholder를 생성합니다.
# 데이터 형태는 [None, 2]과 [None, 1]인데, None은 임의의 행 개수를 의미 -> X는 2개의 변인, y는 1개의 변인
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 2], name="X")
Y = tf.placeholder(tf.float32, shape=[None, 1], name="Y")

In [9]:
# 가중치 변수 W를 생성. 초기값은 정규 분포를 따르는 랜덤한 값임
W = tf.Variable(tf.random_normal([2, 1]), name="weight")

In [10]:
# 편향 변수 b를 생성
b = tf.Variable(tf.random_normal([1]), name="bias")

In [11]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [12]:
# 비용 함수를 설정합니다.(평균 제곱 오차를 계산 )
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [13]:
# 최적화 함수를 설정

# 경사 하강법 최적화기를 생성하고 학습률을 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
# 비용 함수를 최소화하기 위한 훈련 오퍼레이션을 생성
train = optimizer.minimize(cost)

In [14]:
# 세션을 생성
sess = tf.Session()

In [15]:
# 글로벌 변수를 초기화
sess.run(tf.global_variables_initializer())

**학습 수행**
- ※주의※ Nan값 있으면 학습이 제대로 수행안되니 결측치 필수로 제거 후 학습 진행

In [16]:
# 학습을 수행합니다.

for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
        print("#", step, " 손실 비용: ", cost_)
        print("- 하수관로비율(MAX): ", hypo_[0])

# 100001번 반복하는 학습 루프
# 학습을 수행하면서 비용, 가설 값, 훈련을 실행하여 비용 최소화를 진행
# 매 500번째 스텝마다 학습 상황에 대한 내용을 출력

# 0  손실 비용:  104.37206
- 하수관로비율(MAX):  [0.72910243]
# 500  손실 비용:  16.094337
- 하수관로비율(MAX):  [0.7261102]
# 1000  손실 비용:  13.598719
- 하수관로비율(MAX):  [0.72725743]
# 1500  손실 비용:  11.5649
- 하수관로비율(MAX):  [0.7282646]
# 2000  손실 비용:  9.853882
- 하수관로비율(MAX):  [0.729024]
# 2500  손실 비용:  8.414335
- 하수관로비율(MAX):  [0.7295527]
# 3000  손실 비용:  7.2031417
- 하수관로비율(MAX):  [0.72987187]
# 3500  손실 비용:  6.184044
- 하수관로비율(MAX):  [0.72999835]
# 4000  손실 비용:  5.326534
- 하수관로비율(MAX):  [0.72994983]
# 4500  손실 비용:  4.604949
- 하수관로비율(MAX):  [0.72974074]
# 5000  손실 비용:  3.997705
- 하수관로비율(MAX):  [0.72938466]
# 5500  손실 비용:  3.4866421
- 하수관로비율(MAX):  [0.728896]
# 6000  손실 비용:  3.0564919
- 하수관로비율(MAX):  [0.72828484]
# 6500  손실 비용:  2.6944027
- 하수관로비율(MAX):  [0.72756195]
# 7000  손실 비용:  2.389569
- 하수관로비율(MAX):  [0.72673887]
# 7500  손실 비용:  2.1328967
- 하수관로비율(MAX):  [0.7258234]
# 8000  손실 비용:  1.9167411
- 하수관로비율(MAX):  [0.7248242]
# 8500  손실 비용:  1.7346691
- 하수관로비율(MAX):  [0.7237491]
# 9000  손실 비용:  1.5812695
- 하수관로비

**학습 모델 저장**

In [17]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./saved.cpkt")
print('학습된 모델을 저장했습니다.')

# 모델 저장을 위한 Saver 객체를 생성
# 학습된 모델을 저장
# 저장 완료 메시지를 출력

학습된 모델을 저장했습니다.


# [2] 다변인 선형회귀를 활용한 **하수관로 비율 예측**

**Base**

In [18]:
import tensorflow as tf
import numpy as np

In [19]:
# 플레이스 홀더를 설정.
# 입력 데이터와 출력 데이터를 위한 TensorFlow placeholder를 생성.
# 입력 데이터는 [None, 2] 모양이고 출력 데이터는 [None, 1] 모양임.

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [20]:
#가중치와 편향 변수를 생성. 가중치의 모양은 [4, 1]이고 편향의 모양은 [1]

W = tf.Variable(tf.random_normal([2, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

In [21]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [22]:
import tensorflow.compat.v1 as tf
import numpy as np

# 모델 불러오기
sess = tf.Session()
save_path = "./saved.cpkt"
saver = tf.train.import_meta_graph(save_path + ".meta")
saver.restore(sess, save_path)

# 그래프로부터 텐서 가져오기
graph = tf.get_default_graph()
X = graph.get_tensor_by_name("X:0")
hypothesis = graph.get_tensor_by_name("add:0")


In [23]:
# 사용자로부터 강수량을 입력받습니다.

onehr_rain_fall = float(input('1hr 최대 강수량(mm): '))
day_rain_fall = float(input('일평균 강수량(mm): '))

1hr 최대 강수량(mm): 13
일평균 강수량(mm): 80


In [24]:
# 입력 데이터 준비
data = np.array([[onehr_rain_fall, day_rain_fall]], dtype=np.float32)

# 예측 수행
feed_dict = {X: data}
prediction = sess.run(hypothesis, feed_dict)

print("예측된 하수관로비율(MAX):", prediction[0][0])

예측된 하수관로비율(MAX): 0.62453663


# **예측 모델 평가**
- 평균 절대 오차(MAE)가 작을수록 모델의 예측이 실제 데이터와 더 가깝다는 의미
- 결정 계수(R-squared)가 1에 가까울수록 모델이 주어진 데이터를 잘 설명한다는 의미

In [25]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = sess.run(hypothesis, feed_dict={X: x_data})

mae = mean_absolute_error(y_data, y_pred)
r2 = r2_score(y_data, y_pred)

print("평균 절대 오차(MAE):", mae)
print("결정계수(R-squared):", r2)

평균 절대 오차(MAE): 0.33621088
결정계수(R-squared): -0.021881844719134458


## **MSE**

In [30]:
# 예측 수행
feed_dict = {X: x_data}
y_pred = sess.run(hypothesis, feed_dict)

# Mean Squared Error 계산
mse = np.mean(np.square(y_data - y_pred))

# R-squared 계산
total_variance = np.var(y_data)
explained_variance = np.var(y_pred)
r2 = 1 - (mse / total_variance)

print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 0.62792677
